In [1]:
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf

In [2]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
stop_words = stopwords.words('english')

In [3]:
import pandas as pd
training_data = pd.read_csv("../input/train.csv")

In [4]:
training_data['text'] = training_data['keyword'].fillna('') + training_data['location'].fillna('')  \
                        + training_data['text'].fillna('')
training_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
training_data = training_data.drop(columns=['id','keyword','location'],axis=1)
training_data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
def clean_data(text):
    tokens = text.split()
    no_stopwords = [x for x in tokens if x not in stop_words]
    no_nonalphanum = [x.lower() for x in no_stopwords if x.isalnum()]
    return ' '.join(no_nonalphanum)

In [7]:
training_data['text'] = training_data['text'].apply(clean_data)
print(training_data.shape)
training_data.head()

(7613, 2)


,text,target
0,our deeds reason may allah forgive us,1
1,forest fire near la ronge canada,1
2,all residents asked notified no evacuation she...,1
3,people receive evacuation orders california,1
4,just got sent photo ruby smoke pours school,1


In [8]:
test_data = pd.read_csv("../input/test.csv")
test_id = test_data['id']
test_data['text'] = test_data['keyword'].fillna('') + test_data['location'].fillna('')  \
                        + test_data['text'].fillna('')
test_data = test_data.drop(columns=['id','keyword','location'],axis=1)
test_data['text'] = test_data['text'].apply(clean_data)
print(test_data.shape)
test_data.head()

(3263, 1)


,text
0,just happened terrible car crash
1,heard different stay safe
2,forest fire spot geese fleeing across i cannot...
3,apocalypse
4,typhoon soudelor kills 28 china taiwan


In [9]:
model = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
x_train,x_test,y_train,y_test = train_test_split(training_data.text,training_data.target)

In [13]:
history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=512,
                    validation_data=(x_test, y_test),
                    verbose=1)

Train on 5709 samples, validate on 1904 samples
Epoch 1/10
5709/5709 [==============================] - 25s 4ms/sample - loss: 0.6772 - accuracy: 0.5984 - val_loss: 0.6503 - val_accuracy: 0.7096
Epoch 2/10
5709/5709 [==============================] - 23s 4ms/sample - loss: 0.6114 - accuracy: 0.8085 - val_loss: 0.6023 - val_accuracy: 0.7621
Epoch 3/10
5709/5709 [==============================] - 23s 4ms/sample - loss: 0.5384 - accuracy: 0.8469 - val_loss: 0.5504 - val_accuracy: 0.7705
Epoch 4/10
5709/5709 [==============================] - 23s 4ms/sample - loss: 0.4583 - accuracy: 0.8679 - val_loss: 0.5090 - val_accuracy: 0.7736
Epoch 5/10
5709/5709 [==============================] - 23s 4ms/sample - loss: 0.3815 - accuracy: 0.8846 - val_loss: 0.4853 - val_accuracy: 0.7794
Epoch 6/10
5709/5709 [==============================] - 24s 4ms/sample - loss: 0.3130 - accuracy: 0.9058 - val_loss: 0.4808 - val_accuracy: 0.7731
Epoch 7/10
5709/5709 [==============================] - 23s 4ms/sample

In [14]:
results = model.evaluate(x_test,y_test, verbose=2)

1904/1904 - 26s - loss: 0.5597 - accuracy: 0.7537


In [15]:
predictions = model.predict_classes(x_test)
accuracy_score(y_test,predictions)

0.7536764705882353

In [16]:
history = model.fit(training_data.text,
                    training_data.target,
                    epochs=10,
                    batch_size=512,
                    validation_data=(x_test, y_test),
                    verbose=1)

Train on 7613 samples, validate on 1904 samples
Epoch 1/10
7613/7613 [==============================] - 29s 4ms/sample - loss: 0.2257 - accuracy: 0.9187 - val_loss: 0.4401 - val_accuracy: 0.8167
Epoch 2/10
7613/7613 [==============================] - 28s 4ms/sample - loss: 0.1741 - accuracy: 0.9422 - val_loss: 0.2926 - val_accuracy: 0.8892
Epoch 3/10
7613/7613 [==============================] - 29s 4ms/sample - loss: 0.1362 - accuracy: 0.9582 - val_loss: 0.2121 - val_accuracy: 0.9291
Epoch 4/10
7613/7613 [==============================] - 29s 4ms/sample - loss: 0.1113 - accuracy: 0.9683 - val_loss: 0.1571 - val_accuracy: 0.9491
Epoch 5/10
7613/7613 [==============================] - 29s 4ms/sample - loss: 0.0926 - accuracy: 0.9739 - val_loss: 0.1212 - val_accuracy: 0.9632
Epoch 6/10
7613/7613 [==============================] - 29s 4ms/sample - loss: 0.0789 - accuracy: 0.9770 - val_loss: 0.0974 - val_accuracy: 0.9711
Epoch 7/10
7613/7613 [==============================] - 29s 4ms/sample

In [17]:
test_data = test_data['text']
predictions = model.predict_classes(test_data)

In [18]:
predictions = predictions.reshape(-1)
results = pd.DataFrame({
                        'id':test_id,
                        'target':predictions
                        })
results.to_csv('submissions.csv',index=False)

#### Accuracy :---
#####                - Local Accuracy : 75.36 (over split training data)
#####                - Online Accuracy : 74.64 (After fitting over all training data)